In [1]:
import torch
import numpy as np
import torchvision
import torch.nn as nn
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

In [2]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((227, 227)),  # Resize images to 32x32 pixels
    torchvision.transforms.ToTensor(),        # Convert images to PyTorch tensors
    torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,))  # Normalize with mean=0.5, std=0.5
])

train_dataset = torchvision.datasets.CIFAR10(
    root='/home/kami/Documents/datasets/',
    train=True,
    download=False,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
)

for images, labels in train_loader:
    print(f"Batch shape: {images.shape}")  # Should be [64, 1, 32, 32] (batch, channels, height, width)
    print(f"Labels shape: {labels.shape}")  # Should be [64]
    print(f"Image tensor min: {images.min()}, max: {images.max()}")  # Check normalization
    break  # Only print the first batch

Batch shape: torch.Size([64, 3, 227, 227])
Labels shape: torch.Size([64])
Image tensor min: -1.0, max: 1.0


In [3]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10, in_channels=3):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = VGG16(num_classes=10 , in_channels=3).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, weight_decay = 0.005, momentum = 0.9)


st = time.time()

model.train()
for epoch in range(10):
    batch_index = 0
    for images, labels in train_loader:
        # Move data to device
        images = images.to(device)
        labels = labels.to(device)

        # Zero out gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Log loss every 100 batches
        batch_index += 1
        if batch_index % 100 == 0:
            print(f"Epoch: {epoch} | Batch: {batch_index} | Loss: {loss.item():.4f}")

et = time.time()
print(et-st)
print("Training completed.")

Epoch: 0 | Batch: 100 | Loss: 1.8030
Epoch: 0 | Batch: 200 | Loss: 1.5310
Epoch: 0 | Batch: 300 | Loss: 1.3716
Epoch: 0 | Batch: 400 | Loss: 1.5651
Epoch: 0 | Batch: 500 | Loss: 1.1287
Epoch: 0 | Batch: 600 | Loss: 1.2083
Epoch: 0 | Batch: 700 | Loss: 1.0813
Epoch: 1 | Batch: 100 | Loss: 1.0166
Epoch: 1 | Batch: 200 | Loss: 1.0941
Epoch: 1 | Batch: 300 | Loss: 0.9895
Epoch: 1 | Batch: 400 | Loss: 0.7567
Epoch: 1 | Batch: 500 | Loss: 0.6858
Epoch: 1 | Batch: 600 | Loss: 0.9083
Epoch: 1 | Batch: 700 | Loss: 0.9371
Epoch: 2 | Batch: 100 | Loss: 0.7060
Epoch: 2 | Batch: 200 | Loss: 0.7241
Epoch: 2 | Batch: 300 | Loss: 0.6960
Epoch: 2 | Batch: 400 | Loss: 0.7628
Epoch: 2 | Batch: 500 | Loss: 0.9022
Epoch: 2 | Batch: 600 | Loss: 1.0525
Epoch: 2 | Batch: 700 | Loss: 0.5921
Epoch: 3 | Batch: 100 | Loss: 0.5066
Epoch: 3 | Batch: 200 | Loss: 0.5565
Epoch: 3 | Batch: 300 | Loss: 0.4235
Epoch: 3 | Batch: 400 | Loss: 0.9891
Epoch: 3 | Batch: 500 | Loss: 0.6354
Epoch: 3 | Batch: 600 | Loss: 0.8057
E